In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.insert(0,'..')
from datagenerator import jet_data_generator 
import time
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import torch
import os
import pandas as pd

In [3]:
sys.path.insert(0,'../training')
from manifoldembedder import JetDataset, JetPredictDataset
from torch.utils.data import DataLoader, random_split


In [4]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (6,6)
plt.rcParams.update({'font.size': 14})

# Pair Jet Data Generation

In [11]:
np.array([1])/0

/tmp/ipykernel_2378910/3060656171.py:1: RuntimeWarning: divide by zero encountered in true_divide
  np.array([1])/0


array([inf])

In [5]:
np.random.seed()

In [6]:
type_arr = ['2p', '3p']

In [7]:
type_arr[np.random.randint(0,2)]

'2p'

In [8]:
perm = np.random.permutation(10)

In [9]:
perm

array([2, 1, 9, 8, 6, 0, 5, 4, 3, 7])

In [10]:
time.time()

1645220248.6788552

In [11]:
insert = ['qcd','']
print(f'this{insert[1]}')

this


In [12]:
np.random.randint(0,5)

0

In [13]:
os.path.join('')

''

In [6]:
##### savedir   = '/nobackup/users/sangeon/datasets/jetGridpack/jetpairs/'
rawjetdir = '/nobackup/users/sangeon/datasets/jetGridpack/rawjets/'
type_arr  = [['QCD','QCD',''],['2p','twoprong','_80'],['2p','twoprong','_400'],['3p','threeprong','_80'],['3p','threeprong','_400']]
len_train = 1000000
len_half  = 500000
start = time.time()
for jettype in type_arr:


    print(os.path.join(rawjetdir,jettype[0],f'{jettype[1]}{jettype[2]}.h5'))
    df = pd.read_hdf(os.path.join(rawjetdir,jettype[0],f'{jettype[1]}{jettype[2]}.h5'))
    dt = df.values
    dt_train = dt[:1000000]
    dt_val = dt[1000000:1200000]
    dt_predict = dt[1200000:]
    print(dt.shape, dt_train.shape, dt_val.shape, dt_predict.shape)
    column_names = df.columns
    df_train   = pd.DataFrame(dt_train, columns=column_names)
    df_val   = pd.DataFrame(dt_val, columns=column_names)
    df_predict     = pd.DataFrame(dt_predict , columns=column_names)
    df_train.to_hdf(os.path.join(rawjetdir,jettype[0],f'{jettype[1]}{jettype[2]}_train.h5'),"data", mode='w',format='table')
    df_val.to_hdf(os.path.join(rawjetdir,jettype[0],f'{jettype[1]}{jettype[2]}_val.h5'),"data", mode='w',format='table')
    df_predict.to_hdf(os.path.join(rawjetdir,jettype[0],f'{jettype[1]}{jettype[2]}_predict.h5'),"data", mode='w',format='table')

/nobackup/users/sangeon/datasets/jetGridpack/rawjets/QCD/QCD.h5
(1947882, 54) (1000000, 54) (200000, 54) (747882, 54)
/nobackup/users/sangeon/datasets/jetGridpack/rawjets/2p/twoprong_80.h5
(1754310, 54) (1000000, 54) (200000, 54) (554310, 54)
/nobackup/users/sangeon/datasets/jetGridpack/rawjets/2p/twoprong_400.h5
(1345592, 54) (1000000, 54) (200000, 54) (145592, 54)
/nobackup/users/sangeon/datasets/jetGridpack/rawjets/3p/threeprong_80.h5
(2508209, 54) (1000000, 54) (200000, 54) (1308209, 54)
/nobackup/users/sangeon/datasets/jetGridpack/rawjets/3p/threeprong_400.h5
(1382312, 54) (1000000, 54) (200000, 54) (182312, 54)


In [42]:
index = 0

In [5]:
df = pd.read_hdf("/nobackup/users/sangeon/datasets/jetGridpack/rawjets/QCD/QCD_val.h5")

In [6]:
dt = df.values

In [7]:
#np.seterr(all='raise')


{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [7]:
index = -1
def process_jet_data_all(dt, num_part):
    global index
    def fix_phi(phi):
        phi %= (2*np.pi)
        if phi > np.pi:
            phi -= 2*np.pi
        return phi

    def rotate_eig(evt, num_part):
        global index
        index += 1
        #if (index > 26000) & (index < 27000):
            #print(eig_vals)
            #print(index)
        new = np.copy(evt)
        print(evt[:3*num_part].reshape(-1,3)[:, 1:3])
        print(evt[:3*num_part].reshape(-1,3)[:, 0])
        cov_mat = np.cov(evt[:3*num_part].reshape(-1,3)[:, 1:3], aweights=evt[:3*num_part].reshape(-1,3)[:, 0] , rowvar=False)
        #print(new.shape)
        print(cov_mat)
        
        if np.isnan(np.sum(cov_mat)):
            return new
        eig_vals, eig_vecs = np.linalg.eig(cov_mat)
        
        idx = eig_vals.argsort()[::1]   
        eig_vals = eig_vals[idx]
        eig_vecs = eig_vecs[:,idx]
        new[:3*num_part].reshape(-1,3)[:, 1:3] = np.matmul(evt[:3*num_part].reshape(-1,3)[:, 1:3], eig_vecs)
        
        return new

    def flip(evt, num_part):
        global index
        new = np.copy(evt)
        upper_quadrant = np.where(evt[:3*num_part].reshape(-1,3)[:,2]>0)
        lower_quadrant = np.where(evt[:3*num_part].reshape(-1,3)[:,2]<=0)
        upper_sum = np.sum(evt[:3*num_part].reshape(-1,3)[upper_quadrant,0])
        lower_sum = np.sum(evt[:3*num_part].reshape(-1,3)[lower_quadrant,0])
        if lower_sum > upper_sum:
            new[:3*num_part].reshape(-1,3)[:,2] *= -1
        #if index % 1000 == 0:
            #print(index)
        index += 1
        return new

    def flip_eta(evt, num_part):
        global index
        new = np.copy(evt)
        right_quadrant = np.where(evt[:3*num_part].reshape(-1,3)[:,1]>0)
        left_quadrant = np.where(evt[:3*num_part].reshape(-1,3)[:,1]<=0)
        right_sum = np.sum(evt[:3*num_part].reshape(-1,3)[right_quadrant,0])
        left_sum = np.sum(evt[:3*num_part].reshape(-1,3)[left_quadrant,0])
        if left_sum > right_sum:
            new[:3*num_part].reshape(-1,3)[:,1] *= -1
        #if index % 1000 == 0:
            #print(index)
        index += 1
        return new   
    
    temp = np.copy(dt)
    pt = temp[:,3*num_part]
    eta = temp[:,3*num_part+1]
    phi = temp[:,3*num_part+2]
    fix_phi_vec = np.vectorize(fix_phi)
    #print(np.where(pt==0))
    temp[:,:3*num_part].reshape(-1, num_part, 3)[:,:,0] /= pt.reshape(-1,1)
    #print(temp)
    #temp[:,:3*num_part].reshape(-1, num_part, 3)[:,:,0] /= np.sum(temp[:,:3*num_part].reshape(-1, 16, 3)[:,:,0] ,axis=1).reshape(-1,1)
    temp[:,:3*num_part].reshape(-1, num_part, 3)[:,:,1] -= eta.reshape(-1,1)
    temp[:,:3*num_part].reshape(-1, num_part, 3)[:,:,2] = fix_phi_vec(temp[:,:3*num_part].reshape(-1, num_part, 3)[:,:,2] - phi.reshape(-1,1) )
    temp2 = np.apply_along_axis(rotate_eig, 1, temp, num_part)
    index = 0
    temp3 = np.apply_along_axis(flip, 1, temp2, num_part)
    index = 0
    temp4 = np.apply_along_axis(flip_eta, 1, temp3, num_part)
    index = 0
    #temp2 = np.copy(temp)
    return torch.FloatTensor(temp4[:,:3*num_part].reshape(-1, num_part, 3)[:, :, [1, 2, 0]])


In [17]:
dt[0]

array([ 6.05920601e+01,  1.17866457e-01,  5.11139393e-01,  5.53367004e+01,
        1.29850328e-01,  4.81456339e-01,  5.03039284e+01,  1.18905075e-01,
        5.13576090e-01,  3.93967209e+01,  1.81268811e-01,  4.96509016e-01,
        3.73323441e+01,  1.41325578e-01,  4.89765584e-01,  3.54937401e+01,
        1.52147293e-01,  4.43344414e-01,  3.45091934e+01,  1.09749414e-01,
        4.83459294e-01,  3.00410881e+01,  1.12225875e-01,  4.47629243e-01,
        2.71556911e+01,  1.12802595e-01,  4.94287461e-01,  1.70677814e+01,
        2.38600269e-01,  3.51110548e-01,  1.68199043e+01,  4.94805761e-02,
       -2.80564606e-01,  1.60703850e+01,  1.88001320e-01,  4.79018569e-01,
        1.57881041e+01,  2.09237441e-01, -2.75128931e-01,  1.53967028e+01,
        6.20064773e-02,  6.09561980e-01,  1.49796200e+01,  7.53848776e-02,
       -2.32146725e-01,  1.40912046e+01,  2.15860590e-01,  6.17534757e-01,
        6.29240454e+02,  1.40780445e-01,  4.99391809e-01,  8.59949985e+01,
        2.21214665e+02,  

In [9]:
dt[26643]

array([ 7.65304260e+02, -1.64234269e+00, -2.53961420e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        7.65304260e+02, -1.64234269e+00, -2.53961420e+00, -1.86881236e-05,
       -1.86881236e-05, -

In [8]:
process_jet_data_all(dt[26643:26644],16)

[[0.         0.        ]
 [1.64234269 2.5396142 ]
 [1.64234269 2.5396142 ]
 [1.64234269 2.5396142 ]
 [1.64234269 2.5396142 ]
 [1.64234269 2.5396142 ]
 [1.64234269 2.5396142 ]
 [1.64234269 2.5396142 ]
 [1.64234269 2.5396142 ]
 [1.64234269 2.5396142 ]
 [1.64234269 2.5396142 ]
 [1.64234269 2.5396142 ]
 [1.64234269 2.5396142 ]
 [1.64234269 2.5396142 ]
 [1.64234269 2.5396142 ]
 [1.64234269 2.5396142 ]]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[[nan nan]
 [nan nan]]


/tmp/ipykernel_2379677/1030864080.py:19: RuntimeWarning: Degrees of freedom <= 0 for slice
  cov_mat = np.cov(evt[:3*num_part].reshape(-1,3)[:, 1:3], aweights=evt[:3*num_part].reshape(-1,3)[:, 0] , rowvar=False)
/nobackup/users/sangeon/condas/anaconda3/envs/studies/lib/python3.8/site-packages/numpy/lib/function_base.py:2542: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)
/nobackup/users/sangeon/condas/anaconda3/envs/studies/lib/python3.8/site-packages/numpy/lib/function_base.py:2542: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)


tensor([[[-0.0000, -0.0000,  1.0000],
         [-1.6423, -2.5396,  0.0000],
         [-1.6423, -2.5396,  0.0000],
         [-1.6423, -2.5396,  0.0000],
         [-1.6423, -2.5396,  0.0000],
         [-1.6423, -2.5396,  0.0000],
         [-1.6423, -2.5396,  0.0000],
         [-1.6423, -2.5396,  0.0000],
         [-1.6423, -2.5396,  0.0000],
         [-1.6423, -2.5396,  0.0000],
         [-1.6423, -2.5396,  0.0000],
         [-1.6423, -2.5396,  0.0000],
         [-1.6423, -2.5396,  0.0000],
         [-1.6423, -2.5396,  0.0000],
         [-1.6423, -2.5396,  0.0000],
         [-1.6423, -2.5396,  0.0000]]])

In [ ]:
savedir   = '/nobackup/users/sangeon/datasets/jetGridpack/jetpairs/'
rawjetdir = '/nobackup/users/sangeon/datasets/jetGridpack/rawjets/'
type_arr  = [['QCD','QCD',''],['2p','twoprong','_80'],['2p','twoprong','_400'],['3p','threeprong','_80'],['3p','threeprong','_400']]
len_train = 1000000
len_half  = 500000
start = time.time()
for i in range(857,2000):
    np.random.seed()
    type_1 = type_arr[np.random.randint(0,5)]
    np.random.seed()
    type_2 = type_arr[np.random.randint(0,5)]

    
    df1 = pd.read_hdf(os.path.join(rawjetdir,type_1[0],f'{type_1[1]}{type_1[2]}.h5'))
    df2 = pd.read_hdf(os.path.join(rawjetdir,type_2[0],f'{type_2[1]}{type_2[2]}.h5'))
    A = df1.values
    B = df2.values
    
    np.random.seed()
    perm = np.random.permutation(len_train)
    #print(A[perm[:5000]].shape)
    #print(B[perm[5000:]].shape)
    print(os.path.join(rawjetdir,type_1[0],f'{type_1[1]}{type_1[2]}.h5'))
    print(os.path.join(rawjetdir,type_2[0],f'{type_2[1]}{type_2[2]}.h5'))
    
    train_set = JetDataset(False, None, False, A[perm[:10000]],B[perm[len_half:len_half+10000]],16)
    torch.save(train_set, os.path.join(savedir,f'Gridpack_jetpair_train_16part_{i}.pt'))
    
end = time.time()
print('total time', end-start)

/nobackup/users/sangeon/datasets/jetGridpack/rawjets/2p/twoprong_400.h5
/nobackup/users/sangeon/datasets/jetGridpack/rawjets/3p/threeprong_400.h5


100%|██████████| 2/2 [01:31<00:00, 45.52s/it]


/nobackup/users/sangeon/datasets/jetGridpack/rawjets/2p/twoprong_400.h5
/nobackup/users/sangeon/datasets/jetGridpack/rawjets/QCD/QCD.h5


100%|██████████| 2/2 [01:31<00:00, 45.64s/it]


/nobackup/users/sangeon/datasets/jetGridpack/rawjets/2p/twoprong_80.h5
/nobackup/users/sangeon/datasets/jetGridpack/rawjets/2p/twoprong_80.h5


...

100%|██████████| 2/2 [01:30<00:00, 45.47s/it]


/nobackup/users/sangeon/datasets/jetGridpack/rawjets/QCD/QCD.h5
/nobackup/users/sangeon/datasets/jetGridpack/rawjets/3p/threeprong_80.h5


100%|██████████| 2/2 [01:31<00:00, 45.66s/it]


/nobackup/users/sangeon/datasets/jetGridpack/rawjets/2p/twoprong_80.h5
/nobackup/users/sangeon/datasets/jetGridpack/rawjets/2p/twoprong_400.h5


 50%|█████     | 1/2 [00:45<00:45, 45.46s/it]

In [5]:
idx_list = []
for i in range(5):
    for j in range(5):
        idx_list.append([i,j])
        


savedir   = '/nobackup/users/sangeon/datasets/jetGridpack/jetpairs/'
rawjetdir = '/nobackup/users/sangeon/datasets/jetGridpack/rawjets/'
type_arr  = [['QCD','QCD',''],['2p','twoprong','_80'],['2p','twoprong','_400'],['3p','threeprong','_80'],['3p','threeprong','_400']]
len_val = 200000
len_half  = 100000
start = time.time()
for idx in range(25):
    np.random.seed()
    #type_1 = type_arr[np.random.randint(0,5)]
    np.random.seed()
    #type_2 = type_arr[np.random.randint(0,5)]

    type_1 = type_arr[idx_list[idx][0]]
    type_2 = type_arr[idx_list[idx][1]]
    print(type_1, type_2)
    
    df1 = pd.read_hdf(os.path.join(rawjetdir,type_1[0],f'{type_1[1]}{type_1[2]}_val.h5'))
    df2 = pd.read_hdf(os.path.join(rawjetdir,type_2[0],f'{type_2[1]}{type_2[2]}_val.h5'))
    A = df1.values
    B = df2.values
    
    np.random.seed()
    perm = np.random.permutation(len_val)
    #print(A[perm[:5000]].shape)
    #print(B[perm[5000:]].shape)
    print(os.path.join(rawjetdir,type_1[0],f'{type_1[1]}{type_1[2]}_val.h5'))
    print(os.path.join(rawjetdir,type_2[0],f'{type_2[1]}{type_2[2]}_val.h5'))
    
    val_set = JetDataset(False, None, False, A[perm[:len_half]],B[perm[len_half:]],16)
    torch.save(val_set, os.path.join(savedir,f'Gridpack_jetpair_val_16part_{idx}.pt'))
    
end = time.time()
print('total time', end-start)

['QCD', 'QCD', ''] ['QCD', 'QCD', '']
/nobackup/users/sangeon/datasets/jetGridpack/rawjets/QCD/QCD_val.h5
/nobackup/users/sangeon/datasets/jetGridpack/rawjets/QCD/QCD_val.h5


/home/sangeon/ToyJetGenerator/data/../training/manifoldembedder.py:269: RuntimeWarning: Degrees of freedom <= 0 for slice
  cov_mat = np.cov(evt[:3*num_part].reshape(-1,3)[:, 1:3], aweights=evt[:3*num_part].reshape(-1,3)[:, 0] , rowvar=False)
/nobackup/users/sangeon/condas/anaconda3/envs/studies/lib/python3.8/site-packages/numpy/lib/function_base.py:2542: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)
/nobackup/users/sangeon/condas/anaconda3/envs/studies/lib/python3.8/site-packages/numpy/lib/function_base.py:2542: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)
100%|██████████| 20/20 [12:55<00:00, 38.78s/it]


['QCD', 'QCD', ''] ['2p', 'twoprong', '_80']
/nobackup/users/sangeon/datasets/jetGridpack/rawjets/QCD/QCD_val.h5
/nobackup/users/sangeon/datasets/jetGridpack/rawjets/2p/twoprong_80_val.h5


100%|██████████| 20/20 [12:48<00:00, 38.45s/it]


['QCD', 'QCD', ''] ['2p', 'twoprong', '_400']
/nobackup/users/sangeon/datasets/jetGridpack/rawjets/QCD/QCD_val.h5
/nobackup/users/sangeon/datasets/jetGridpack/rawjets/2p/twoprong_400_val.h5


...

['3p', 'threeprong', '_400'] ['3p', 'threeprong', '_80']
/nobackup/users/sangeon/datasets/jetGridpack/rawjets/3p/threeprong_400_val.h5
/nobackup/users/sangeon/datasets/jetGridpack/rawjets/3p/threeprong_80_val.h5


100%|██████████| 20/20 [15:10<00:00, 45.53s/it]


['3p', 'threeprong', '_400'] ['3p', 'threeprong', '_400']
/nobackup/users/sangeon/datasets/jetGridpack/rawjets/3p/threeprong_400_val.h5
/nobackup/users/sangeon/datasets/jetGridpack/rawjets/3p/threeprong_400_val.h5


100%|██████████| 20/20 [15:10<00:00, 45.53s/it]

total time 22581.080144643784


In [6]:
rawjetdir = '/nobackup/users/sangeon/datasets/jets/rawjets/realistic'
with open(os.path.join(rawjetdir,f'realistic_rawjet_test_1p_16part_merged.pickle'), 'rb') as handle:
    A = pickle.load(handle)

In [7]:
A.shape

(200000, 48)

In [ ]:
savedir   = '/nobackup/users/sangeon/datasets/jets/jetpairs/realistic'
rawjetdir = '/nobackup/users/sangeon/datasets/jets/rawjets/realistic'
type_arr  = ['1p','2p','3p']
len_train = 1000000
len_half  = 500000
start = time.time()
for i in range(22,100):
    np.random.seed()
    type_1 = type_arr[np.random.randint(0,3)]
    np.random.seed()
    type_2 = type_arr[np.random.randint(0,3)]
    np.random.seed()
    with open(os.path.join(rawjetdir,f'realistic_rawjet_train_{type_1}_16part_merged.pickle'), 'rb') as handle:
        A = pickle.load(handle)
    with open(os.path.join(rawjetdir,f'realistic_rawjet_train_{type_2}_16part_merged.pickle'), 'rb') as handle:
        B = pickle.load(handle)
    np.random.seed()
    perm = np.random.permutation(len_train)
    #print(A[perm[:5000]].shape)
    #print(B[perm[5000:]].shape)
    train_set = JetDataset(False, None, True, A[perm[:len_half]],B[perm[len_half:]],16)
    torch.save(train_set, os.path.join(savedir,f'realistic_jetpair_train_16part_{i}.pt'))
    
end = time.time()
print('total time', end-start)

 58%|█████▊    | 58/100 [37:42<27:19, 39.04s/it] 

In [8]:
index_list = []
for i in range(3):
    for j in range(3):
        index_list.append([i,j])

In [10]:
index_list

[[0, 0], [0, 1], [0, 2], [1, 0], [1, 1], [1, 2], [2, 0], [2, 1], [2, 2]]

In [11]:
savedir   = '/nobackup/users/sangeon/datasets/jets/jetpairs/realistic'
rawjetdir = '/nobackup/users/sangeon/datasets/jets/rawjets/realistic'
type_arr  = ['1p','2p','3p']
len_val = 200000
len_half  = 100000
start = time.time()
for i, idx in enumerate(index_list):
    np.random.seed()
    type_1 = type_arr[idx[0]]
    np.random.seed()
    type_2 = type_arr[idx[1]]
    np.random.seed()
    print(type_1, type_2)
    with open(os.path.join(rawjetdir,f'realistic_rawjet_val_{type_1}_16part_merged.pickle'), 'rb') as handle:
        A = pickle.load(handle)
    with open(os.path.join(rawjetdir,f'realistic_rawjet_val_{type_2}_16part_merged.pickle'), 'rb') as handle:
        B = pickle.load(handle)
    np.random.seed()
    perm = np.random.permutation(len_val)
    #print(A[perm[:5000]].shape)
    #print(B[perm[5000:]].shape)
    val_set = JetDataset(False, None, True, A[perm[:len_half]],B[perm[len_half:]],16)
    torch.save(val_set, os.path.join(savedir,f'realistic_jetpair_val_16part_{i}.pt'))
    
end = time.time()
print('total time', end-start)

1p 3p


100%|██████████| 20/20 [13:08<00:00, 39.44s/it]


2p 1p


100%|██████████| 20/20 [12:59<00:00, 39.00s/it]


2p 2p


...

3p 2p


100%|██████████| 20/20 [12:57<00:00, 38.87s/it]


3p 3p


100%|██████████| 20/20 [12:58<00:00, 38.91s/it]

total time 5464.729719877243


In [5]:
index_list = []
for i in range(4):
    for j in range(4):
        index_list.append([i,j])

In [6]:
savedir   = '/nobackup/users/sangeon/datasets/jets/jetpairs/realistic'
rawjetdir = '/nobackup/users/sangeon/datasets/jets/rawjets/realistic'
type_arr  = ['1p','2p','3p','4p']
len_test = 200000
len_half  = 100000
start = time.time()
for i, idx in enumerate(index_list):
    np.random.seed()
    type_1 = type_arr[idx[0]]
    np.random.seed()
    type_2 = type_arr[idx[1]]
    np.random.seed()
    print(type_1, type_2)
    with open(os.path.join(rawjetdir,f'realistic_rawjet_test_{type_1}_16part_merged.pickle'), 'rb') as handle:
        A = pickle.load(handle)
    with open(os.path.join(rawjetdir,f'realistic_rawjet_test_{type_2}_16part_merged.pickle'), 'rb') as handle:
        B = pickle.load(handle)
    np.random.seed()
    perm = np.random.permutation(len_test)
    #print(A[perm[:5000]].shape)
    #print(B[perm[5000:]].shape)
    test_set = JetDataset(False, None, True, A[perm[:len_half]],B[perm[len_half:]],16)
    torch.save(test_set, os.path.join(savedir,f'realistic_jetpair_test_16part_{i}.pt'))
    
end = time.time()
print('total time', end-start)

1p 1p


100%|██████████| 20/20 [13:08<00:00, 39.40s/it]


1p 2p


100%|██████████| 20/20 [12:52<00:00, 38.60s/it]


1p 3p


...

4p 3p


100%|██████████| 20/20 [12:56<00:00, 38.82s/it]


4p 4p


100%|██████████| 20/20 [12:52<00:00, 38.64s/it]

total time 12395.912010908127


In [ ]:
savedir   = '/nobackup/users/sangeon/datasets/jets/jetpairs/simple'
rawjetdir = '/nobackup/users/sangeon/datasets/jets/rawjets/simple'
type_arr  = ['1p','2p','3p']
len_train = 2000000
len_half  = 1000000
start = time.time()
for i in range(115,1000):
    np.random.seed()
    type_1 = type_arr[np.random.randint(0,3)]
    np.random.seed()
    type_2 = type_arr[np.random.randint(0,3)]
    np.random.seed()
    with open(os.path.join(rawjetdir,f'simple_rawjet_train_{type_1}_16part_merged.pickle'), 'rb') as handle:
        A = pickle.load(handle)
    with open(os.path.join(rawjetdir,f'simple_rawjet_train_{type_2}_16part_merged.pickle'), 'rb') as handle:
        B = pickle.load(handle)
    np.random.seed()
    perm = np.random.permutation(len_train)
    #print(A[perm[:5000]].shape)
    #print(B[perm[5000:]].shape)
    train_set = JetDataset(False, None, True, A[perm[:100000]],B[perm[len_half:len_half+100000]],16)
    torch.save(train_set, os.path.join(savedir,f'simple_jetpair_train_16part_{i}.pt'))
    
end = time.time()
print('total time', end-start)

 40%|████      | 8/20 [06:23<09:54, 49.54s/it]

In [5]:
rawjetdir = '/nobackup/users/sangeon/datasets/jets/rawjets/realistic'
with open(os.path.join(rawjetdir,f'realistic_rawjet_val_2p_16part_merged.pickle'), 'rb') as handle:
     A = pickle.load(handle)
print(A.shape)

(200000, 48)


In [8]:
rawjetdir = '/nobackup/users/sangeon/datasets/jets/rawjets/simple'
with open(os.path.join(rawjetdir,f'simple_rawjet_val_2p_16part_merged.pickle'), 'rb') as handle:
     A = pickle.load(handle)
print(A.shape)

(400000, 48)


In [9]:
savedir   = '/nobackup/users/sangeon/datasets/jets/jetpairs/simple'
rawjetdir = '/nobackup/users/sangeon/datasets/jets/rawjets/simple'
type_arr  = ['1p','2p','3p']
len_val = 400000
len_half  = 200000
start = time.time()
for i in range(0,100):
    np.random.seed()
    type_1 = type_arr[np.random.randint(0,3)]
    np.random.seed()
    type_2 = type_arr[np.random.randint(0,3)]
    np.random.seed()
    with open(os.path.join(rawjetdir,f'simple_rawjet_val_{type_1}_16part_merged.pickle'), 'rb') as handle:
        A = pickle.load(handle)
    with open(os.path.join(rawjetdir,f'simple_rawjet_val_{type_2}_16part_merged.pickle'), 'rb') as handle:
        B = pickle.load(handle)
    np.random.seed()
    perm = np.random.permutation(len_val)
    #print(A[perm[:5000]].shape)
    #print(B[perm[5000:]].shape)
    train_set = JetDataset(False, None, True, A[perm[:len_half]],B[perm[len_half:]],16)
    torch.save(train_set, os.path.join(savedir,f'simple_jetpair_val_16part_{i}.pt'))
    
end = time.time()
print('total time', end-start)

  2%|▎         | 1/40 [00:28<18:24, 28.31s/it]


KeyboardInterrupt: 

# Predict, Test Dataset

In [6]:
rawjetdir = '/nobackup/users/sangeon/datasets/jetGridpack/rawjets/'
type_arr  = [['QCD','QCD',''],['2p','twoprong','_80_predict'],['2p','twoprong','_400_predict'],['3p','threeprong','_80_predict'],['3p','threeprong','_400_predict'],['4p','fourprong','_170'],['4p','fourprong','_400']]
set_list = []
name_list = ['QCD','2p25','2p170','3p25','3p170','4p170','4p400']

results = np.empty((0,54))
predict_label = np.empty((0))
#extrafeats = np.empty((0,14))
for i, (name, jettype) in enumerate(zip(name_list, type_arr)):
    df = pd.read_hdf(os.path.join(rawjetdir,jettype[0],f'{jettype[1]}{jettype[2]}.h5'))
    print(df.values.shape)
    data = df.values[:,:54]
    #feats = df.values[:,48:]
    #print(feats.shape)
    #feats = feats[:100000]
    #extrafeats = np.vstack([extrafeats, feats])
    data = data[:100000]
    results = np.vstack([results, data])
    print(df.values.shape[0])
    print(name, jettype)
    predict_label = np.concatenate([predict_label, np.repeat(i, data.shape[0])])

(1947882, 54)
1947882
QCD ['QCD', 'QCD', '']
(554310, 54)
554310
2p25 ['2p', 'twoprong', '_80_predict']
(145592, 54)
145592
2p170 ['2p', 'twoprong', '_400_predict']
(1308209, 54)
1308209
3p25 ['3p', 'threeprong', '_80_predict']
(182312, 54)
182312
3p170 ['3p', 'threeprong', '_400_predict']
(207456, 54)
207456
4p170 ['4p', 'fourprong', '_170']
(187644, 54)
187644
4p400 ['4p', 'fourprong', '_400']


In [7]:
results.shape

(700000, 54)

In [8]:
predict_set = JetPredictDataset(False,None,False, results,predict_label,16)
torch.save(predict_set, f'/nobackup/users/sangeon/datasets/jetGridpack/jetpairs/Gridpack_jetpair_secondensemble_predict.pt')

In [6]:
rawjetdir = '/nobackup/users/sangeon/datasets/jetGridpack/rawjets/'
type_arr  = [['QCD/with_taus','QCD',''],['2p/with_taus','twoprong','_25'],['2p/with_taus','twoprong','_170'],['3p/with_taus','threeprong','_25'],['3p/with_taus','threeprong','_170'],['4p/with_taus','fourprong','_170'],['4p/with_taus','fourprong','_400']]
set_list = []
name_list = ['QCD','2p25','2p170','3p25','3p170','4p170','4p400']

results = np.empty((0,54))
predict_label = np.empty((0))
extrafeats = np.empty((0,14))
for i, (name, jettype) in enumerate(zip(name_list, type_arr)):
    df = pd.read_hdf(os.path.join(rawjetdir,jettype[0],f'{jettype[1]}{jettype[2]}.h5'))
    print(df.values.shape)
    data = df.values[:,:54]
    feats = df.values[:,48:]
    print(feats.shape)
    feats = feats[:100000]
    extrafeats = np.vstack([extrafeats, feats])
    data = data[:100000]
    results = np.vstack([results, data])
    print(df.values.shape[0])
    print(name, jettype)
    predict_label = np.concatenate([predict_label, np.repeat(i, data.shape[0])])

(1951762, 62)
(1951762, 14)
1951762
QCD ['QCD/with_taus', 'QCD', '']
(922324, 62)
(922324, 14)
922324
2p25 ['2p/with_taus', 'twoprong', '_25']
(803501, 62)
(803501, 14)
803501
2p170 ['2p/with_taus', 'twoprong', '_170']
(318466, 62)
(318466, 14)
318466
3p25 ['3p/with_taus', 'threeprong', '_25']
(278615, 62)
(278615, 14)
278615
3p170 ['3p/with_taus', 'threeprong', '_170']
(153516, 62)
(153516, 14)
153516
4p170 ['4p/with_taus', 'fourprong', '_170']
(127011, 62)
(127011, 14)
127011
4p400 ['4p/with_taus', 'fourprong', '_400']


In [7]:
extrafeats.shape

(700000, 14)

In [8]:
results.shape

(700000, 54)

In [9]:
predict_label

array([0., 0., 0., ..., 6., 6., 6.])

In [10]:
with open(f'/nobackup/users/sangeon/datasets/jetGridpack/jetpairs/taus_Gridpack_jetpair_predict_extrafeats.pickle','wb') as handle:
    pickle.dump(extrafeats, handle, pickle.HIGHEST_PROTOCOL)

In [5]:
idx_list = []
for i in range(7):
    for j in range(7):
        idx_list.append([i,j])
        


savedir   = '/nobackup/users/sangeon/datasets/jetGridpack/jetpairs/'
rawjetdir = '/nobackup/users/sangeon/datasets/jetGridpack/rawjets/'
type_arr  = [['QCD','QCD',''],['2p','twoprong','_25'],['2p','twoprong','_170'],['3p','threeprong','_25'],['3p','threeprong','_170'],['4p','fourprong','_170'],['4p','fourprong','_400']]
len_test = 100000
len_half  = 50000
start = time.time()
for idx in range(11,49):
    np.random.seed()
    #type_1 = type_arr[np.random.randint(0,5)]
    np.random.seed()
    #type_2 = type_arr[np.random.randint(0,5)]

    type_1 = type_arr[idx_list[idx][0]]
    type_2 = type_arr[idx_list[idx][1]]
    print(type_1, type_2)
    
    df1 = pd.read_hdf(os.path.join(rawjetdir,type_1[0],f'{type_1[1]}{type_1[2]}.h5'))
    df2 = pd.read_hdf(os.path.join(rawjetdir,type_2[0],f'{type_2[1]}{type_2[2]}.h5'))
    A = df1.values[:len_test]
    B = df2.values[:len_test]
    
    np.random.seed()
    perm = np.random.permutation(len_test)
    #print(A[perm[:5000]].shape)
    #print(B[perm[5000:]].shape)
    print(os.path.join(rawjetdir,type_1[0],f'{type_1[1]}{type_1[2]}.h5'))
    print(os.path.join(rawjetdir,type_2[0],f'{type_2[1]}{type_2[2]}.h5'))
    
    test_set = JetDataset(False, None, False, A[perm[:len_half]],B[perm[len_half:]],16)
    torch.save(test_set, os.path.join(savedir,f'Gridpack_jetpair_testDistortion_16part_{idx}.pt'))
    
end = time.time()
print('total time', end-start)

['2p', 'twoprong', '_25'] ['3p', 'threeprong', '_170']
/nobackup/users/sangeon/datasets/jetGridpack/rawjets/2p/twoprong_25.h5
/nobackup/users/sangeon/datasets/jetGridpack/rawjets/3p/threeprong_170.h5


/home/sangeon/ToyJetGenerator/data/../training/manifoldembedder.py:195: RuntimeWarning: Degrees of freedom <= 0 for slice
  cov_mat = np.cov(evt[:3*num_part].reshape(-1,3)[:, 1:3], aweights=evt[:3*num_part].reshape(-1,3)[:, 0] , rowvar=False)
/nobackup/users/sangeon/condas/anaconda3/envs/studies/lib/python3.8/site-packages/numpy/lib/function_base.py:2680: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)
/nobackup/users/sangeon/condas/anaconda3/envs/studies/lib/python3.8/site-packages/numpy/lib/function_base.py:2680: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)
100%|██████████| 10/10 [06:41<00:00, 40.11s/it]


['2p', 'twoprong', '_25'] ['4p', 'fourprong', '_170']
/nobackup/users/sangeon/datasets/jetGridpack/rawjets/2p/twoprong_25.h5
/nobackup/users/sangeon/datasets/jetGridpack/rawjets/4p/fourprong_170.h5


100%|██████████| 10/10 [06:26<00:00, 38.63s/it]


['2p', 'twoprong', '_25'] ['4p', 'fourprong', '_400']
/nobackup/users/sangeon/datasets/jetGridpack/rawjets/2p/twoprong_25.h5
/nobackup/users/sangeon/datasets/jetGridpack/rawjets/4p/fourprong_400.h5


...

['4p', 'fourprong', '_400'] ['4p', 'fourprong', '_170']
/nobackup/users/sangeon/datasets/jetGridpack/rawjets/4p/fourprong_400.h5
/nobackup/users/sangeon/datasets/jetGridpack/rawjets/4p/fourprong_170.h5


100%|██████████| 10/10 [06:26<00:00, 38.63s/it]


['4p', 'fourprong', '_400'] ['4p', 'fourprong', '_400']
/nobackup/users/sangeon/datasets/jetGridpack/rawjets/4p/fourprong_400.h5
/nobackup/users/sangeon/datasets/jetGridpack/rawjets/4p/fourprong_400.h5


100%|██████████| 10/10 [06:26<00:00, 38.70s/it]

total time 16050.89022231102


In [11]:
results.shape

(700000, 54)

In [12]:
predict_label.shape

(700000,)

In [13]:
predict_set = JetPredictDataset(False,None,False, results,predict_label,16)
torch.save(predict_set, f'/nobackup/users/sangeon/datasets/jetGridpack/jetpairs/taus_Gridpack_jetpair_predict.pt')

In [14]:
torch.save(predict_set, f'/nobackup/users/sangeon/datasets/jetGridpack/jetpairs/taus_Gridpack_jetpair_predict.pt')

In [8]:
rawjetdir = '/nobackup/users/sangeon/datasets/jetGridpack/rawjets/'
type_arr  = [['QCD','QCD',''],['2p','twoprong','_80'],['2p','twoprong','_400'],['3p','threeprong','_80'],['3p','threeprong','_400']]
set_list = []
name_list = ['QCD','2p80','2p400','3p80','3p400']

results = np.empty((0,54))
predict_label = np.empty((0))
#extrafeats = np.empty((0,14))
for i, (name, jettype) in enumerate(zip(name_list, type_arr)):
    df = pd.read_hdf(os.path.join(rawjetdir,jettype[0],f'{jettype[1]}{jettype[2]}_predict.h5'))
    print(df.values.shape)
    data = df.values[:,:54]
    #feats = df.values[:,48:]
    #print(feats.shape)
    #feats = feats[:100000]
    #extrafeats = np.vstack([extrafeats, feats])
    data = data[:100000]
    results = np.vstack([results, data])
    print(df.values.shape[0])
    print(name, jettype)
    predict_label = np.concatenate([predict_label, np.repeat(i, data.shape[0])])

(747882, 54)
747882
QCD ['QCD', 'QCD', '']
(554310, 54)
554310
2p80 ['2p', 'twoprong', '_80']
(145592, 54)
145592
2p400 ['2p', 'twoprong', '_400']
(1308209, 54)
1308209
3p80 ['3p', 'threeprong', '_80']
(182312, 54)
182312
3p400 ['3p', 'threeprong', '_400']


In [9]:
results.shape

(500000, 54)

In [11]:
predict_label.shape

(500000,)

In [12]:
predict_set = JetPredictDataset(False,None,False, results,predict_label,16)


/home/sangeon/ToyJetGenerator/data/../training/manifoldembedder.py:372: RuntimeWarning: Degrees of freedom <= 0 for slice
  cov_mat = np.cov(evt[:3*num_part].reshape(-1,3)[:, 1:3], aweights=evt[:3*num_part].reshape(-1,3)[:, 0] , rowvar=False)
/nobackup/users/sangeon/condas/anaconda3/envs/studies/lib/python3.8/site-packages/numpy/lib/function_base.py:2680: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)
/nobackup/users/sangeon/condas/anaconda3/envs/studies/lib/python3.8/site-packages/numpy/lib/function_base.py:2680: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)


In [13]:
torch.save(predict_set, f'/nobackup/users/sangeon/datasets/jetGridpack/jetpairs/interpolate_Gridpack_jetpair_predict.pt')

In [5]:
idx_list = []
for i in range(5):
    for j in range(5):
        idx_list.append([i,j])
        


savedir   = '/nobackup/users/sangeon/datasets/jetGridpack/jetpairs/'
rawjetdir = '/nobackup/users/sangeon/datasets/jetGridpack/rawjets/'
type_arr  = [['QCD','QCD',''],['2p','twoprong','_80'],['2p','twoprong','_400'],['3p','threeprong','_80'],['3p','threeprong','_400']]
len_test = 100000
len_half  = 50000
start = time.time()
for idx in range(25):
    np.random.seed()
    #type_1 = type_arr[np.random.randint(0,5)]
    np.random.seed()
    #type_2 = type_arr[np.random.randint(0,5)]

    type_1 = type_arr[idx_list[idx][0]]
    type_2 = type_arr[idx_list[idx][1]]
    print(type_1, type_2)
    
    df1 = pd.read_hdf(os.path.join(rawjetdir,type_1[0],f'{type_1[1]}{type_1[2]}_predict.h5'))
    df2 = pd.read_hdf(os.path.join(rawjetdir,type_2[0],f'{type_2[1]}{type_2[2]}_predict.h5'))
    A = df1.values[:len_test]
    B = df2.values[:len_test]
    
    np.random.seed()
    perm = np.random.permutation(len_test)
    #print(A[perm[:5000]].shape)
    #print(B[perm[5000:]].shape)
    print(os.path.join(rawjetdir,type_1[0],f'{type_1[1]}{type_1[2]}_predict.h5'))
    print(os.path.join(rawjetdir,type_2[0],f'{type_2[1]}{type_2[2]}_predict.h5'))
    
    test_set = JetDataset(False, None, False, A[perm[:len_half]],B[perm[len_half:]],16)
    torch.save(test_set, os.path.join(savedir,f'Gridpack_jetpair_interpolateDistortion_16part_{idx}.pt'))
    
end = time.time()
print('total time', end-start)

['QCD', 'QCD', ''] ['QCD', 'QCD', '']
/nobackup/users/sangeon/datasets/jetGridpack/rawjets/QCD/QCD.h5
/nobackup/users/sangeon/datasets/jetGridpack/rawjets/QCD/QCD.h5


/home/sangeon/ToyJetGenerator/data/../training/manifoldembedder.py:211: RuntimeWarning: Degrees of freedom <= 0 for slice
  cov_mat = np.cov(evt[:3*num_part].reshape(-1,3)[:, 1:3], aweights=evt[:3*num_part].reshape(-1,3)[:, 0] , rowvar=False)
/nobackup/users/sangeon/condas/anaconda3/envs/studies/lib/python3.8/site-packages/numpy/lib/function_base.py:2680: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)
/nobackup/users/sangeon/condas/anaconda3/envs/studies/lib/python3.8/site-packages/numpy/lib/function_base.py:2680: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)
100%|██████████| 10/10 [06:47<00:00, 40.79s/it]


['QCD', 'QCD', ''] ['2p', 'twoprong', '_80']
/nobackup/users/sangeon/datasets/jetGridpack/rawjets/QCD/QCD.h5
/nobackup/users/sangeon/datasets/jetGridpack/rawjets/2p/twoprong_80.h5


100%|██████████| 10/10 [07:06<00:00, 42.65s/it]


['QCD', 'QCD', ''] ['2p', 'twoprong', '_400']
/nobackup/users/sangeon/datasets/jetGridpack/rawjets/QCD/QCD.h5
/nobackup/users/sangeon/datasets/jetGridpack/rawjets/2p/twoprong_400.h5


...

['3p', 'threeprong', '_400'] ['3p', 'threeprong', '_80']
/nobackup/users/sangeon/datasets/jetGridpack/rawjets/3p/threeprong_400.h5
/nobackup/users/sangeon/datasets/jetGridpack/rawjets/3p/threeprong_80.h5


100%|██████████| 10/10 [06:40<00:00, 40.05s/it]


['3p', 'threeprong', '_400'] ['3p', 'threeprong', '_400']
/nobackup/users/sangeon/datasets/jetGridpack/rawjets/3p/threeprong_400.h5
/nobackup/users/sangeon/datasets/jetGridpack/rawjets/3p/threeprong_400.h5


100%|██████████| 10/10 [06:39<00:00, 39.96s/it]

total time 11121.34701347351


In [6]:
print('yes')

yes


In [5]:
predict_label

NameError: name 'predict_label' is not defined

In [19]:
results.shape

(700000, 48)

In [20]:
predict_label.shape

(700000,)

In [17]:
results.shape

(1387644, 48)

In [5]:
with open(f'/nobackup/users/sangeon/datasets/jets/rawjets/realistic/realistic_rawjet_test_1p_16part_merged.pickle', 'rb') as handle:
    onep = pickle.load(handle)
with open(f'/nobackup/users/sangeon/datasets/jets/rawjets/realistic/realistic_rawjet_test_2p_16part_merged.pickle', 'rb') as handle:
    twop = pickle.load(handle)
with open(f'/nobackup/users/sangeon/datasets/jets/rawjets/realistic/realistic_rawjet_test_3p_16part_merged.pickle', 'rb') as handle:
    threep = pickle.load(handle)
with open(f'/nobackup/users/sangeon/datasets/jets/rawjets/realistic/realistic_rawjet_test_4p_16part_merged.pickle', 'rb') as handle:
    fourp = pickle.load(handle)

In [7]:
print(onep.shape, twop.shape, threep.shape, fourp.shape)

(200000, 48) (200000, 48) (200000, 48) (200000, 48)


In [10]:
predictdata = np.vstack([onep,twop,threep,fourp])

predict_label = np.concatenate([np.ones(200000),np.ones(200000)*2,np.ones(200000)*3,np.ones(200000)*4])

predict_set = JetPredictDataset(False,None,predictdata,predict_label,16)

torch.save(predict_set, f'/nobackup/users/sangeon/datasets/jets/jetpairs/realistic/realistic_jetpredict_16part.pt')

# Theta Z information save

In [6]:
with open(f'/nobackup/users/sangeon/datasets/jets/rawjets/realistic/realistic_rawjet_zthetatest_1p_16part_merged_data.pickle', 'rb') as handle:
    onep = pickle.load(handle)
with open(f'/nobackup/users/sangeon/datasets/jets/rawjets/realistic/realistic_rawjet_zthetatest_2p_16part_merged_data.pickle', 'rb') as handle:
    twop = pickle.load(handle)
with open(f'/nobackup/users/sangeon/datasets/jets/rawjets/realistic/realistic_rawjet_zthetatest_3p_16part_merged_data.pickle', 'rb') as handle:
    threep = pickle.load(handle)
with open(f'/nobackup/users/sangeon/datasets/jets/rawjets/realistic/realistic_rawjet_zthetatest_4p_16part_merged_data.pickle', 'rb') as handle:
    fourp = pickle.load(handle)

In [5]:
with open(f'/nobackup/users/sangeon/datasets/jets/rawjets/realistic/realistic_rawjet_zthetatest_1p_16part_merged_z.pickle', 'rb') as handle:
    onep_z = pickle.load(handle)
with open(f'/nobackup/users/sangeon/datasets/jets/rawjets/realistic/realistic_rawjet_zthetatest_2p_16part_merged_z.pickle', 'rb') as handle:
    twop_z = pickle.load(handle)
with open(f'/nobackup/users/sangeon/datasets/jets/rawjets/realistic/realistic_rawjet_zthetatest_3p_16part_merged_z.pickle', 'rb') as handle:
    threep_z = pickle.load(handle)
with open(f'/nobackup/users/sangeon/datasets/jets/rawjets/realistic/realistic_rawjet_zthetatest_4p_16part_merged_z.pickle', 'rb') as handle:
    fourp_z = pickle.load(handle)

In [6]:
with open(f'/nobackup/users/sangeon/datasets/jets/rawjets/realistic/realistic_rawjet_zthetatest_1p_16part_merged_theta.pickle', 'rb') as handle:
    onep_theta = pickle.load(handle)
with open(f'/nobackup/users/sangeon/datasets/jets/rawjets/realistic/realistic_rawjet_zthetatest_2p_16part_merged_theta.pickle', 'rb') as handle:
    twop_theta = pickle.load(handle)
with open(f'/nobackup/users/sangeon/datasets/jets/rawjets/realistic/realistic_rawjet_zthetatest_3p_16part_merged_theta.pickle', 'rb') as handle:
    threep_theta = pickle.load(handle)
with open(f'/nobackup/users/sangeon/datasets/jets/rawjets/realistic/realistic_rawjet_zthetatest_4p_16part_merged_theta.pickle', 'rb') as handle:
    fourp_theta = pickle.load(handle)

In [12]:
print(onep.shape, twop.shape, threep.shape, fourp.shape)

(200000, 48) (200000, 48) (200000, 48) (200000, 48)


In [14]:
predictdata = np.vstack([onep,twop,threep,fourp])
print(predictdata.shape)
predict_label = np.concatenate([np.ones(200000),np.ones(200000)*2,np.ones(200000)*3,np.ones(200000)*4])

predict_set = JetPredictDataset(False,None,True,predictdata,predict_label,16)

torch.save(predict_set, f'/nobackup/users/sangeon/datasets/jets/jetpairs/realistic/realistic_jetpredict_zthetatest_16part.pt')

(800000, 48)


In [7]:
predict_z = np.vstack([onep_z,twop_z,threep_z,fourp_z])


In [8]:
predict_theta = np.vstack([onep_theta,twop_theta,threep_theta,fourp_theta])


In [9]:
with open(f'/nobackup/users/sangeon/datasets/jets/jetpairs/realistic/realistic_jetpredict_zthetatest_16part_z.pickle', 'wb') as handle:
    pickle.dump(predict_z, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [10]:
with open(f'/nobackup/users/sangeon/datasets/jets/jetpairs/realistic/realistic_jetpredict_zthetatest_16part_theta.pickle', 'wb') as handle:
    pickle.dump(predict_theta, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [8]:
with open(f'/nobackup/users/sangeon/datasets/jets/rawjets/simple/simple_rawjet_test_1p_16part_merged.pickle', 'rb') as handle:
    onep = pickle.load(handle)
with open(f'/nobackup/users/sangeon/datasets/jets/rawjets/simple/simple_rawjet_test_2p_16part_merged.pickle', 'rb') as handle:
    twop = pickle.load(handle)
with open(f'/nobackup/users/sangeon/datasets/jets/rawjets/simple/simple_rawjet_test_3p_16part_merged.pickle', 'rb') as handle:
    threep = pickle.load(handle)
with open(f'/nobackup/users/sangeon/datasets/jets/rawjets/simple/simple_rawjet_test_4p_16part_merged.pickle', 'rb') as handle:
    fourp = pickle.load(handle)

In [9]:
onep.shape

(400000, 48)

In [10]:
twop.shape

(400000, 48)

In [12]:
threep.shape

(400000, 48)

In [13]:
fourp.shape

(400000, 48)

In [15]:
predictdata = np.vstack([onep,twop,threep,fourp])

In [16]:
predictdata.shape

(1600000, 48)

In [17]:
predict_label = np.concatenate([np.ones(400000),np.ones(400000)*2,np.ones(400000)*3,np.ones(400000)*4])

In [19]:
predict_label.shape

(1600000,)

In [20]:
predict_set = JetPredictDataset(False,None,predictdata,predict_label,16)

torch.save(predict_set, f'/nobackup/users/sangeon/datasets/jets/jetpairs/simple/simple_jetpredict_16part.pt')

In [ ]:
predictdata = np.zeros((200000, 48))
with open(f'/nobackup/users/sangeon/datasets/jets/rawjets/rawjet_16p_test_top_2p_{i}.pickle', 'rb') as handle:
    A = pickle.load(handle)
    predictdata[i*10000:(i+1)*10000] = A
        
with open(f'/nobackup/users/sangeon/datasets/jets/rawjets/rawjet_16p_test_top_3p_{i}.pickle', 'rb') as handle:
    A = pickle.load(handle)
    predictdata[100000+i*10000:100000+(i+1)*10000] = A        
        
        

In [10]:
predictdata = np.zeros((200000, 48))
for i in range(10):
    with open(f'/nobackup/users/sangeon/datasets/jets/rawjets/rawjet_16p_test_top_2p_{i}.pickle', 'rb') as handle:
        A = pickle.load(handle)
        predictdata[i*10000:(i+1)*10000] = A
        
for i in range(10):
    with open(f'/nobackup/users/sangeon/datasets/jets/rawjets/rawjet_16p_test_top_3p_{i}.pickle', 'rb') as handle:
        A = pickle.load(handle)
        predictdata[100000+i*10000:100000+(i+1)*10000] = A        
        
        

In [10]:
predictdata = np.zeros((200000, 24))
for i in range(10):
    with open(f'../data/rawjet_predict_top_2p_{i}.pickle', 'rb') as handle:
        A = pickle.load(handle)
        predictdata[i*10000:(i+1)*10000] = A
        
for i in range(10):
    with open(f'../data/rawjet_predict_top_3p_{i}.pickle', 'rb') as handle:
        A = pickle.load(handle)
        predictdata[100000+i*10000:100000+(i+1)*10000] = A        
        
        

In [11]:
predict_label = np.concatenate([np.zeros(100000),np.ones(100000)])

In [15]:
predictdata[0].reshape(-1,3)

array([[ 6.99495974e+01, -4.62456322e-01,  9.37513138e-03],
       [ 2.08527162e+01, -5.08091968e-01, -1.37167399e-02],
       [ 9.45262145e+00, -1.69237894e-01,  8.44849371e-01],
       [ 1.31046161e+02, -1.86486084e-01, -1.64772091e-01],
       [ 1.40813906e+01, -1.87280071e-01, -1.65739820e-01],
       [ 2.74764412e+00, -2.10888479e-01, -1.20393760e-01],
       [ 2.87590590e+01, -1.70390329e-01,  8.25092420e-01],
       [ 3.96745565e+01, -1.80208756e-01, -1.08734917e-01],
       [ 1.10372210e+01, -5.00814653e-01, -2.06670027e-02],
       [ 2.58505190e+01, -4.99693093e-01, -2.14799391e-02],
       [ 1.19876449e-01, -2.07820133e-01,  7.23484579e-01],
       [ 1.41887849e+00, -2.47684380e-01,  3.82181474e-01],
       [ 6.44399335e+01, -4.60593627e-01,  5.88519453e-03],
       [ 4.33020192e-02, -2.85940766e-01,  8.85119217e-01],
       [ 5.30016595e+00, -4.60519062e-01,  6.48273680e-03],
       [ 2.09678433e-01, -2.74214655e-01,  3.94282241e-01]])

In [14]:
#predict_set = JetPredictDataset(False,None,predictdata,predict_label,8)
predict_set = JetPredictDataset(False,None,predictdata,predict_label,16)

torch.save(predict_set, f'/nobackup/users/sangeon/datasets/jets/jet_16p_predict_temp.pt')

In [11]:
for i in range(100):
    np.random.seed()
    type_1 = type_arr[np.random.randint(0,2)]
    type_2 = type_arr[np.random.randint(0,2)]
    np.random.seed()
    index_1 = np.random.randint(0,10)
    index_2 = np.random.randint(0,10)
    with open(f'rawjet_test_top_{type_1}_{index_1}.pickle', 'rb') as handle:
        A = pickle.load(handle)
    with open(f'rawjet_test_top_{type_2}_{index_2}.pickle', 'rb') as handle:
        B = pickle.load(handle)
    np.random.seed()
    perm = np.random.permutation(10000)
    #print(A[perm[:5000]].shape)
    #print(B[perm[5000:]].shape)
    test_set = JetDataset(False, None, A[perm[:5000]],B[perm[5000:]],8)
    torch.save(test_set, f'/nobackup/users/sangeon/datasets/jets/jet_test_8part_{i}.pt')


100%|██████████| 40/40 [00:11<00:00,  3.51it/s]


In [10]:
for i in range(1000):
    np.random.seed()
    type_1 = type_arr[np.random.randint(0,2)]
    type_2 = type_arr[np.random.randint(0,2)]
    np.random.seed()
    index_1 = np.random.randint(0,100)
    index_2 = np.random.randint(0,100)
    with open(f'rawjet_val_top_{type_1}_{index_1}.pickle', 'rb') as handle:
        A = pickle.load(handle)
    with open(f'rawjet_val_top_{type_2}_{index_2}.pickle', 'rb') as handle:
        B = pickle.load(handle)
    np.random.seed()
    perm = np.random.permutation(10000)
    #print(A[perm[:5000]].shape)
    #print(B[perm[5000:]].shape)
    val_set = JetDataset(False, None, A[perm[:5000]],B[perm[5000:]],8)
    torch.save(val_set, f'/nobackup/users/sangeon/datasets/jets/jet_val_8part_{i}.pt')


100%|██████████| 40/40 [00:11<00:00,  3.50it/s]


In [7]:
import os

In [9]:
basedir = '/nobackup/users/sangeon/datasets/jets/rawjets'
for i in range(0,200):
    np.random.seed()
    type_1 = type_arr[np.random.randint(0,2)]
    type_2 = type_arr[np.random.randint(0,2)]
    np.random.seed()
    index_1 = np.random.randint(0,100)
    index_2 = np.random.randint(0,100)
    with open(os.path.join(basedir,f'rawjet_16p_val_top_{type_1}_{index_1}.pickle'), 'rb') as handle:
        A = pickle.load(handle)
    with open(os.path.join(basedir,f'rawjet_16p_val_top_{type_2}_{index_2}.pickle'), 'rb') as handle:
        B = pickle.load(handle)
    np.random.seed()
    perm = np.random.permutation(10000)
    #print(A[perm[:5000]].shape)
    #print(B[perm[5000:]].shape)
    val_set = JetDataset(False, None, A[perm[:5000]],B[perm[5000:]],16)
    torch.save(val_set, f'/nobackup/users/sangeon/datasets/jets/jet_val_16part_{i}.pt')


100%|██████████| 40/40 [00:50<00:00,  1.26s/it]


In [10]:
basedir = '/nobackup/users/sangeon/datasets/jets/rawjets'
for i in range(0,200):
    np.random.seed()
    type_1 = type_arr[np.random.randint(0,2)]
    type_2 = type_arr[np.random.randint(0,2)]
    np.random.seed()
    index_1 = np.random.randint(0,70)
    index_2 = np.random.randint(0,70)
    with open(os.path.join(basedir,f'rawjet_16p_test_top_{type_1}_{index_1}.pickle'), 'rb') as handle:
        A = pickle.load(handle)
    with open(os.path.join(basedir,f'rawjet_16p_test_top_{type_2}_{index_2}.pickle'), 'rb') as handle:
        B = pickle.load(handle)
    np.random.seed()
    perm = np.random.permutation(10000)
    #print(A[perm[:5000]].shape)
    #print(B[perm[5000:]].shape)
    test_set = JetDataset(False, None, A[perm[:5000]],B[perm[5000:]],16)
    torch.save(test_set, f'/nobackup/users/sangeon/datasets/jets/jet_test_16part_{i}.pt')


100%|██████████| 40/40 [00:50<00:00,  1.26s/it]


In [ ]:
basedir = '/nobackup/users/sangeon/datasets/jets/rawjets'
for i in range(245,1000):
    np.random.seed()
    type_1 = type_arr[np.random.randint(0,2)]
    type_2 = type_arr[np.random.randint(0,2)]
    np.random.seed()
    index_1 = np.random.randint(0,274)
    index_2 = np.random.randint(0,274)
    with open(os.path.join(basedir,f'rawjet_16p_train_top_{type_1}_{index_1}.pickle'), 'rb') as handle:
        A = pickle.load(handle)
    with open(os.path.join(basedir,f'rawjet_16p_train_top_{type_2}_{index_2}.pickle'), 'rb') as handle:
        B = pickle.load(handle)
    np.random.seed()
    perm = np.random.permutation(10000)
    #print(A[perm[:5000]].shape)
    #print(B[perm[5000:]].shape)
    train_set = JetDataset(False, None, A[perm[:5000]],B[perm[5000:]],16)
    torch.save(train_set, f'/nobackup/users/sangeon/datasets/jets/jet_train_16part_{i}.pt')


 62%|██████▎   | 25/40 [00:27<00:16,  1.08s/it]

In [ ]:
for i in range(2236,5000):
    np.random.seed()
    type_1 = type_arr[np.random.randint(0,2)]
    type_2 = type_arr[np.random.randint(0,2)]
    np.random.seed()
    index_1 = np.random.randint(0,600)
    index_2 = np.random.randint(0,600)
    with open(f'rawjet_train_top_{type_1}_{index_1}.pickle', 'rb') as handle:
        A = pickle.load(handle)
    with open(f'rawjet_train_top_{type_2}_{index_2}.pickle', 'rb') as handle:
        B = pickle.load(handle)
    np.random.seed()
    perm = np.random.permutation(10000)
    #print(A[perm[:5000]].shape)
    #print(B[perm[5000:]].shape)
    train_set = JetDataset(False, None, A[perm[:5000]],B[perm[5000:]],8)
    torch.save(train_set, f'/nobackup/users/sangeon/datasets/jets/jet_train_8part_{i}.pt')


 85%|████████▌ | 34/40 [00:09<00:01,  3.45it/s]

In [ ]:
for i in tqdm(range(100)):
    start = time.time()
    np.random.seed()
    top_2p = top_2p_8part.generate_dataset(10000)
    np.random.seed()
    top_3p = top_3p_8part.generate_dataset(10000)
    with open(f'rawjet_train_top_2p_{i}.pickle', 'wb') as handle:
        pickle.dump(top_2p, handle, protocol=pickle.HIGHEST_PROTOCOL)
    with open(f'rawjet_train_top_3p_{i}.pickle', 'wb') as handle:
        pickle.dump(top_3p, handle, protocol=pickle.HIGHEST_PROTOCOL)
    end = time.time()
    print("time: ", end-start)

  1%|          | 1/100 [01:12<2:00:10, 72.83s/it]

time:  72.82877159118652


  2%|▏         | 2/100 [02:25<1:59:09, 72.96s/it]

time:  73.04729223251343


  3%|▎         | 3/100 [03:39<1:58:06, 73.06s/it]

...

time:  73.48512578010559


 96%|█████████▌| 96/100 [1:57:14<04:53, 73.33s/it]

time:  73.25415897369385


 97%|█████████▋| 97/100 [1:58:27<03:40, 73.35s/it]

time:  73.37689065933228


In [7]:
for i in tqdm(range(587,600)):
    start = time.time()
    np.random.seed()
    top_2p = top_2p_8part.generate_dataset(10000)
    np.random.seed()
    top_3p = top_3p_8part.generate_dataset(10000)
    with open(f'rawjet_train_top_2p_{i}.pickle', 'wb') as handle:
        pickle.dump(top_2p, handle, protocol=pickle.HIGHEST_PROTOCOL)
    with open(f'rawjet_train_top_3p_{i}.pickle', 'wb') as handle:
        pickle.dump(top_3p, handle, protocol=pickle.HIGHEST_PROTOCOL)
    end = time.time()
    print("time: ", end-start)

  8%|▊         | 1/13 [01:17<15:28, 77.40s/it]

time:  77.39782905578613


 15%|█▌        | 2/13 [02:36<14:24, 78.60s/it]

time:  79.44325947761536


 23%|██▎       | 3/13 [03:55<13:04, 78.41s/it]

...

time:  77.62972283363342


 92%|█████████▏| 12/13 [15:42<01:18, 78.42s/it]

time:  78.15016460418701


100%|██████████| 13/13 [17:02<00:00, 78.67s/it]

time:  80.07684516906738


In [7]:
for i in tqdm(range(100)):
    start = time.time()
    np.random.seed()
    top_2p = top_2p_8part.generate_dataset(10000)
    np.random.seed()
    top_3p = top_3p_8part.generate_dataset(10000)
    with open(f'rawjet_val_top_2p_{i}.pickle', 'wb') as handle:
        pickle.dump(top_2p, handle, protocol=pickle.HIGHEST_PROTOCOL)
    with open(f'rawjet_val_top_3p_{i}.pickle', 'wb') as handle:
        pickle.dump(top_3p, handle, protocol=pickle.HIGHEST_PROTOCOL)
    end = time.time()
    print("time: ", end-start)

  1%|          | 1/100 [01:32<2:33:15, 92.89s/it]

time:  92.88565015792847


  2%|▏         | 2/100 [03:04<2:30:16, 92.01s/it]

time:  91.38690519332886


  3%|▎         | 3/100 [04:37<2:29:39, 92.57s/it]

...

time:  91.71151399612427


 99%|█████████▉| 99/100 [2:31:46<01:33, 93.14s/it]

time:  91.91695261001587


100%|██████████| 100/100 [2:33:20<00:00, 92.01s/it]

time:  94.62391686439514


In [17]:
import os

In [18]:
train_file_list = []
val_file_list = []
test_file_list = []
predict_file_list = []

for file in os.listdir('/nobackup/users/sangeon/datasets/jets'):
    if "16part" in file:
        if "train" in file:
            train_file_list.append(os.path.join('/nobackup/users/sangeon/datasets/jets',file))
        elif "val" in file:
            val_file_list.append(os.path.join('/nobackup/users/sangeon/datasets/jets',file))
        #elif "predict" in file:
        #    predict_file_list.append(os.path.join('/nobackup/users/sangeon/datasets/jets',file))
        elif "test" in file:
            test_file_list.append(os.path.join('/nobackup/users/sangeon/datasets/jets',file))

train_list = []
for file in train_file_list:
    train_list.append(torch.load(file))
jet_train = torch.utils.data.ConcatDataset(train_list)
torch.save(jet_train, f'/nobackup/users/sangeon/datasets/jets/concat_16p_train_temp.pt')

val_list = []
for file in val_file_list:
    val_list.append(torch.load(file))
jet_val = torch.utils.data.ConcatDataset(val_list)
torch.save(jet_val, f'/nobackup/users/sangeon/datasets/jets/concat_16p_val_temp.pt')

test_list = []
for file in test_file_list:
    test_list.append(torch.load(file))
jet_test = torch.utils.data.ConcatDataset(test_list)
torch.save(jet_test, f'/nobackup/users/sangeon/datasets/jets/concat_16p_test_temp.pt')

#predict_list = []
#for file in predict_file_list:
#    predict_list.append(torch.load(file))
#jet_predict = torch.utils.data.ConcatDataset(predict_list)
#torch.save(jet_predict, f'/nobackup/users/sangeon/datasets/jets/concat_predict_temp.pt')

In [17]:
top_2p[np.random.permutation(5)][0]

array([ 1.42852400e+02, -9.90205463e-02, -1.35309180e-01,  1.05521532e+01,
       -3.94737006e-01,  7.89277596e-01,  3.09849542e+01, -3.91857433e-01,
        7.91430808e-01,  5.41413282e+01, -9.89971734e-02, -1.36130549e-01,
        9.89379634e+01, -5.29106113e-01, -7.09260387e-03,  1.94524879e+01,
       -5.80055975e-01,  1.44811893e-02,  3.46883336e+01, -5.79956566e-01,
        1.45399891e-02,  1.96171629e+01, -5.29102045e-01, -7.04731236e-03])

In [ ]:
for i in tqdm(range(10)):
    start = time.time()
    
    top_2p = top_2p_8part.generate_dataset(10000)
    top_3p = top_3p_8part.generate_dataset(10000)
    np.random.seed()

    
    

    trainindices_a = torch.tensor(np.random.randint(len(traindataset), size=10000))
    trainindices_b = torch.tensor(np.random.randint(len(traindataset), size=10000))
    a = torch.FloatTensor(traindataset.data[trainindices_a]/255.)
    b = torch.FloatTensor(traindataset.data[trainindices_b]/255.)
    
    train_set = MNISTDataset("POT",False, None, a, b)
    torch.save(train_set, f'/nobackup/users/sangeon/datasets/mnist/trainset_{i}.pt')
    
    end = time.time()
    print(end-start)